# Webscrapeo : Ubica tu casilla

### Paquetes

In [1]:
import requests
import numpy as np
import pandas as pd

### Pipeline

In [5]:
API = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo-with-cmr-uts?e=1&s=1"
headers = {"authority": "us-central1-ine-ubica-tu-casilla.cloudfunctions.net",
            "method": "GET",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br",
            "origin": "https://ubicatucasilla.ine.mx",
            "referer": "https://ubicatucasilla.ine.mx/",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36",
            "x-utc-service-id": "ubicatucasilla",
            "x-utc-version-id": "2022"}

In [16]:
r = requests.get(API, headers=headers)
content = r.json()['data'][0]['items']
content[0]

{'code': '03',
 'seccionSede': 1,
 'entidadSede': 1,
 'distritoSede': 3,
 'distrito_localSede': 6,
 'utSede': 64,
 'secciones': ['0001'],
 'apellidos': [{'casilla': 'B',
   'inicial': 'ABOYTES DOÑAS',
   'final': 'LLAMAS DELGADO'},
  {'casilla': 'C1', 'inicial': 'LLAMAS ESPERON', 'final': 'ZUÑIGA GONZALEZ'}],
 'reference': {'dce': {'geom': {'coordinates': [-102.321935418, 21.929536847],
    'type': 'Point'},
   'ref': {'distrito_federal': 3,
    'distrito_local': 6,
    'entidad': 1,
    'seccion': 1},
   'version': '2021'}},
 'data': {'ut': 64,
  'active': True,
  'domicilio': 'CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL CAMPESTRE SEGUNDA SECCIÓN, 20100, AGUASCALIENTES, AGUASCALIENTES',
  'referencia': 'ENTRE CALLE FRONTERA Y CALLE MEZQUITE',
  'ubicacion': 'DOMICILIO PARTICULAR',
  'dce': {'coordinates': [-102.321935418, 21.929536847], 'type': 'Point'},
  'geom': {'coordinates': [-102.321935418, 21.929536847], 'type': 'Point'}}}

In [27]:
raw_df = pd.DataFrame(content[0]['apellidos'])
raw_df['domicilio'] = content[0]['data']['domicilio']
raw_df['domicilio'] = content[0]['data']['domicilio']

raw_df['ubicacion'] = content[0]['data']['ubicacion']

raw_df['entidad'] = content[0]['entidadSede']
raw_df['seccion'] = content[0]['seccionSede']


raw_df

,casilla,inicial,final,domicilio,ubicacion,entidad,seccion
0,B,ABOYTES DOÑAS,LLAMAS DELGADO,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,1
1,C1,LLAMAS ESPERON,ZUÑIGA GONZALEZ,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,1
